# Práctica 3


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
df = pd.read_csv("diabetes_dataset.csv")
df_2019 = df[df["year"] == 2019]
df_2019.drop(columns=["year"], inplace=True)
df_2019.head()

C:\Users\jlflo\AppData\Local\Temp\ipykernel_18800\1727960749.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019.drop(columns=["year"], inplace=True)


,gender,age,location,race:AfricanAmerican,race:Asian,race:Caucasian,race:Hispanic,race:Other,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
73,Male,55.00,Alabama,0,0,0,1,0,0,0,No Info,27.94,6.0,158,0
335,Female,1.72,Alabama,0,0,0,1,0,0,0,No Info,16.28,6.2,100,0
337,Female,55.00,Alabama,0,1,0,0,0,0,0,never,28.30,6.0,140,0
340,Female,53.00,Alabama,0,1,0,0,0,0,0,never,35.84,6.0,155,0
341,Female,38.00,Alabama,0,1,0,0,0,0,0,never,31.91,5.8,200,0


In [11]:
cols_discretizar = {
    "age": {
        "niños": [0, 9],
        "adolescentes": [10, 19],
        "jóvenes": [20, 59],
        "adulto mayor": [60],
    },
    "bmi": {
        "bajo peso": [18.49],
        "peso normal": [18.5, 24.99],
        "sobrepeso": [25, 29.99],
        "obesidad": [30],
    },
    "hbA1c_level": {"normal": [5.6], "limite": [5.7, 6.4], "sobre": [6.5]},
    "blood_glucose_level": {
        "hipoglucemia": [79],
        "normal": [80, 159],
        "elevado": [160, 199],
        "critico": [200],
    },
}


def discretizar_col(col, bins: dict):
    factor = 0
    for label, edges in bins.items():
        if len(edges) == 1:
            if factor == 0:
                if col <= edges[0]:
                    return label

            else:
                if col >= edges[0]:
                    return label
        else:
            if edges[0] <= col <= edges[1]:
                return label
        factor += 1


def discretizar_df(df: pd.DataFrame, cols_dict: dict) -> pd.DataFrame:
    df_copy = df.copy()
    for col, bins in cols_dict.items():
        df_copy[col] = df_copy[col].apply(lambda x: discretizar_col(x, bins))
    return df_copy


df_discretizado = discretizar_df(df_2019, cols_discretizar)
df_discretizado.head()

,gender,age,location,race:AfricanAmerican,race:Asian,race:Caucasian,race:Hispanic,race:Other,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
73,Male,jóvenes,Alabama,0,0,0,1,0,0,0,No Info,sobrepeso,limite,normal,0
335,Female,niños,Alabama,0,0,0,1,0,0,0,No Info,bajo peso,limite,normal,0
337,Female,jóvenes,Alabama,0,1,0,0,0,0,0,never,sobrepeso,limite,normal,0
340,Female,jóvenes,Alabama,0,1,0,0,0,0,0,never,obesidad,limite,normal,0
341,Female,jóvenes,Alabama,0,1,0,0,0,0,0,never,obesidad,limite,critico,0


In [12]:
vals_numericos = {
    "age": {
        "niños": 0,
        "adolescentes": 1,
        "jóvenes": 2,
        "adulto mayor": 3,
    },
    "bmi": {
        "bajo peso": 0,
        "peso normal": 1,
        "sobrepeso": 2,
        "obesidad": 3,
    },
    "hbA1c_level": {"normal": 0, "limite": 1, "sobre": 2},
    "blood_glucose_level": {
        "hipoglucemia": 0,
        "normal": 1,
        "elevado": 2,
        "critico": 3,
    },
    "gender": {"Male": 0, "Female": 1},
}


def dummy_col(col, vals: dict):
    for label, val in vals.items():
        if col == label:
            return int(val)


def dummy_df(df: pd.DataFrame, cols_discretizar: dict) -> pd.DataFrame:
    for col, bins in cols_discretizar.items():
        df[col] = df[col].apply(lambda x: dummy_col(x, bins))
    for col in df.columns:
        # Verificar si la columna necesita conversión
        necesita_conversion = False

        if df[col].dtype == "object":
            # Es texto, necesita conversión
            necesita_conversion = True
        elif df[col].dtype in ["float64", "float32", "int64", "int32"]:
            # Es numérica, verificar si tiene valores no enteros
            try:
                valores_no_nulos = df[col].dropna()
                if len(valores_no_nulos) > 0:
                    # Verificar si todos los valores son enteros
                    necesita_conversion = not all(
                        valores_no_nulos == valores_no_nulos.astype(int)
                    )
            except:
                necesita_conversion = True

        if necesita_conversion:
            # Obtener valores únicos (excluyendo NaN)
            valores_unicos = df[col].dropna().unique()

            # Crear el mapeo (ordenado alfabéticamente)
            mapeo = {
                valor: idx for idx, valor in enumerate(sorted(valores_unicos, key=str))
            }

            # Guardar en el diccionario
            cols_discretizar[col] = mapeo

            # Aplicar la conversión
            df[col] = df[col].map(mapeo)

    return df, cols_discretizar


df_dummy, vals_numericos = dummy_df(df_discretizado, vals_numericos)
df_dummy.head()

,gender,age,location,race:AfricanAmerican,race:Asian,race:Caucasian,race:Hispanic,race:Other,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
73,0.0,2,0,0,0,0,1,0,0,0,0,2,1,1,0
335,1.0,0,0,0,0,0,1,0,0,0,0,0,1,1,0
337,1.0,2,0,0,1,0,0,0,0,0,4,2,1,1,0
340,1.0,2,0,0,1,0,0,0,0,0,4,3,1,1,0
341,1.0,2,0,0,1,0,0,0,0,0,4,3,1,3,0


In [13]:
vals_numericos

{'age': {'niños': 0, 'adolescentes': 1, 'jóvenes': 2, 'adulto mayor': 3},
 'bmi': {'bajo peso': 0, 'peso normal': 1, 'sobrepeso': 2, 'obesidad': 3},
 'hbA1c_level': {'normal': 0, 'limite': 1, 'sobre': 2},
 'blood_glucose_level': {'hipoglucemia': 0,
  'normal': 1,
  'elevado': 2,
  'critico': 3},
 'gender': {'Male': 0, 'Female': 1},
 'location': {'Alabama': 0,
  'Alaska': 1,
  'Arizona': 2,
  'Arkansas': 3,
  'California': 4,
  'Colorado': 5,
  'Connecticut': 6,
  'Delaware': 7,
  'District of Columbia': 8,
  'Florida': 9,
  'Georgia': 10,
  'Guam': 11,
  'Hawaii': 12,
  'Idaho': 13,
  'Illinois': 14,
  'Indiana': 15,
  'Iowa': 16,
  'Kansas': 17,
  'Kentucky': 18,
  'Louisiana': 19,
  'Maine': 20,
  'Maryland': 21,
  'Massachusetts': 22,
  'Michigan': 23,
  'Minnesota': 24,
  'Mississippi': 25,
  'Missouri': 26,
  'Montana': 27,
  'Nebraska': 28,
  'Nevada': 29,
  'New Hampshire': 30,
  'New Jersey': 31,
  'New Mexico': 32,
  'New York': 33,
  'North Carolina': 34,
  'North Dakota': 35

In [14]:
def generar_poblacion(df_original: pd.DataFrame, N: int = 120) -> np.ndarray:
    """
    Genera una población aleatoria a partir de una base de datos original.
    args:
        df_original (pd.DataFrame): DataFrame original del cual se generará la población.
        N (int): Tamaño de la población a generar. Default es 120.
    returns:
        np.ndarray: Arreglo con la población generada.
    """
    n_df = pd.DataFrame(columns=df_original.columns)
    for col in df_original.columns:
        n_df[col] = np.random.choice(df_original[col], size=N, replace=True)
    poblacion = np.zeros((N, 2, len(n_df.columns)))
    for i in range(len(poblacion)):
        poblacion[i][0] = np.random.choice([0, 1, 2], size=n_df.shape[1])
        poblacion[i][1] = n_df.iloc[i].values

    return poblacion

In [ ]:
from math import comb


def calcular_H(N, M):
    """
    Calcula H dado N (puntos deseados) y M (objetivos)
    mediante búsqueda iterativa.
    """
    H = 1
    while comb(H + M - 1, M - 1) < N:
        H += 1

    puntos_generados = comb(H + M - 1, M - 1)

    if puntos_generados == N:
        return H
    else:
        print(
            f"No existe H exacto. H={H} genera {puntos_generados} puntos (cercano a {N})"
        )
        return H


def Dan_Dennis(poblacion: np.ndarray, M=4) -> np.ndarray:
    """
    Ejecuta el método de generación de población de Dan Dennis.
    args:
        poblacion (np.ndarray): Matriz original del cual se generará la población.
        M (int): Número de objetivos.
    returns:
        np.ndarray: Arreglo con los pesos.
    """
    N = poblacion.shape[0]
    H = calcular_H(N, M)
    # print(f"Usando H={H} para generar la población.")

    def generar_particiones(M, H, profundidad=0, suma_actual=0, particion_actual=[]):
        """
        Genera recursivamente todas las particiones que suman H.
        """
        if profundidad == M - 1:
            # Última posición: asignar lo que falta
            particion_actual.append(H - suma_actual)
            particiones.append(particion_actual[:])
            particion_actual.pop()
            return

        # Probar todos los valores posibles para la posición actual
        for valor in range(H - suma_actual + 1):
            particion_actual.append(valor)
            generar_particiones(
                M, H, profundidad + 1, suma_actual + valor, particion_actual
            )
            particion_actual.pop()

    particiones = []
    generar_particiones(M, H)

    # Convertir particiones a pesos
    weights = np.array(particiones, dtype=float) / H

    return weights

In [18]:
from typing import Callable

In [19]:
def evaluar_pesos(
    individuo: np.ndarray, pesos: np.ndarray, obtener_objetivos: Callable
) -> float:
    """
    Evalúa un individuo según los pesos dados.
    Retorna el valor evaluado.
    """
    valor = 0
    objetivos = obtener_objetivos(individuo)
    for i in range(len(objetivos)):
        valor += objetivos[i] * pesos[i]
    return valor


def comparar_individuos(
    ind1: np.ndarray, ind2: np.ndarray, pesos: np.ndarray, obtener_objetivos: Callable
) -> int:
    """
    Compara dos individuos según los pesos dados.
    Retorna:
        -1 si ind1 es mejor que ind2
         1 si ind2 es mejor que ind1
         0 si son iguales
    """
    val_ind1 = evaluar_pesos(ind1, pesos, obtener_objetivos)
    val_ind2 = evaluar_pesos(ind2, pesos, obtener_objetivos)

    if val_ind1 > val_ind2:
        return -1
    elif val_ind2 > val_ind1:
        return 1
    else:
        return 0

# Funciones Objetivo


## Funciones principales


In [ ]:
def obtener_x_y(individuo: np.ndarray):
    index = individuo[0]
    x = []
    y = []
    for i in range(len(index)):
        if index[i] == 1:
            x.append(i)
        elif index[i] == 2:
            x.append(i)
    return x, y


def obtener_matriz_confusion(individuo: np.ndarray, df: pd.DataFrame):
    x, y = obtener_x_y(individuo)
    cols = df.columns
    a, b, c, d = 0, 0, 0, 0
    for i in range(len(df)):
        aux_y = 0
        for j in range(len(y)):
            if df[cols[y[j]]][i] == individuo[1][y[j]]:
                aux_y += 1
        aux_x = 0
        for j in range(len(x)):
            if df[cols[x[j]]][i] == individuo[1][x[j]]:
                aux_x += 1
        if aux_y == len(y) and aux_x == len(x):
            a += 1
        elif aux_y == len(y) and aux_x != len(x):
            b += 1
        elif aux_y != len(y) and aux_x == len(x):
            c += 1
        else:
            d += 1
    return [[a, b], [c, d]]

In [ ]:
def support(a: int, N: int) -> float:
    """
    Calcula el soporte de un individuo.
    args:
        a (int): X ∩ Y.
        N (int): Número total de instancias.
    returns:
        float: Soporte del individuo.
    """

    return a / N


def confidence(a: int, c: int) -> float:
    """
    Calcula la confianza de un individuo.
    args:
        a (int): X ∩ Y.
        c (int): X ∩ ~Y.
    returns:
        float: Confianza del individuo.
    """

    if a + c == 0:
        return 0.0
    return a / (a + c)


def lift(a: int, b: int, c: int, N: int) -> float:
    """
    Calcula el lift de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Lift del individuo.
    """

    if (a + b) * (a + c) == 0:
        return 0.0
    return (a * N) / ((a + b) * (a + c))


def conviction(a: int, b: int, c: int, d: int, N: int) -> float:
    """
    Calcula la convicción de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        d (int): ~X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Convicción del individuo.
    """

    den = 1 - a / (a + c)
    if den == 0:
        return float("inf")
    return (1 - (a + b) / N) / den

## Problema de Optimización 1


In [ ]:
def casual_support(a: int, d: int, N: int) -> float:
    """
    Calcula el soporte causal.
    args:
        a (int): X ∩ Y.
        d (int): ~X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Soporte causal del individuo.
    """

    return (a + d) / N


def casual_confidence(a: int, b: int, c: int, d: int) -> float:
    """
    Calcula la confianza causal.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        d (int): ~X ∩ ~Y.
    returns:
        float: Confianza causal del individuo.
    """

    conf_xy = a / (a + c) if (a + c) > 0 else 0.0

    conf_notx_noty = d / (b + d) if (b + d) > 0 else 0.0

    return 0.5 * (conf_xy + conf_notx_noty)


def max_confidence(a: int, b: int, c: int) -> float:
    """
    Calcula la confianza máxima.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        d (int): ~X ∩ ~Y.
    returns:
        float: Confianza máxima del individuo.
    """

    conf_x_to_y = a / (a + c) if (a + c) > 0 else 0.0

    conf_y_to_x = a / (a + b) if (a + b) > 0 else 0.0

    return max(conf_x_to_y, conf_y_to_x)

## Problema de Optimización 2


In [ ]:
def jaccard(a: int, b: int, c: int, N: int) -> float:
    """
    Calcula el coeficiente de Jaccard de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Coeficiente de Jaccard del individuo.
    """

    p_xy = a / N
    p_x = (a + c) / N
    p_y = (a + b) / N

    denominador = p_x + p_y - p_xy

    if denominador == 0:
        return 0.0

    return p_xy / denominador


def cosine(a: int, b: int, c: int, N: int) -> float:
    """
    Calcula la similitud coseno de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Similitud coseno del individuo.
    """

    p_x = (a + c) / N
    p_y = (a + b) / N

    if p_x * p_y == 0:
        return 0.0

    p_xy = a / N

    return p_xy / (p_x * p_y) ** 0.5


def phi_coefficient(a: int, b: int, c: int, N: int) -> float:
    """
    Calcula el coeficiente phi de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Coeficiente phi del individuo.
    """

    p_xy = a / N

    p_x = (a + c) / N
    p_y = (a + b) / N

    numerador = p_xy - p_x * p_y
    denominador = (p_x * (1 - p_x) * p_y * (1 - p_y)) ** 0.5

    if denominador == 0:
        return 0.0

    return numerador / denominador


def kappa(a: int, b: int, c: int, d: int, N: int) -> float:
    """
    Calcula el coeficiente kappa de un individuo.
    args:
        a (int): X ∩ Y.
        b (int): ~X ∩ Y.
        c (int): X ∩ ~Y.
        d (int): ~X ∩ ~Y.
        N (int): Número total de instancias.
    returns:
        float: Coeficiente kappa del individuo.
    """

    p_xy = a / N

    p_notx_noty = d / N

    p_x = (a + c) / N
    p_y = (a + b) / N
    p_notx = (b + d) / N
    p_noty = (c + d) / N

    numerador = p_xy + p_notx_noty - p_x * p_y - p_notx * p_noty
    denominador = 1 - p_x * p_y - p_notx * p_noty

    if denominador == 0:
        return 0.0

    return numerador / denominador

## Función general


In [ ]:
def obtener_objetivos(
    individuo: np.ndarray, df: pd.DataFrame, problema: int = 0
) -> np.ndarray:
    matriz_confusion = obtener_matriz_confusion(individuo, df)
    a = matriz_confusion[0][0]
    b = matriz_confusion[0][1]
    c = matriz_confusion[1][0]
    d = matriz_confusion[1][1]
    N = df.shape[0]
    if problema == 0:
        return np.array(
            [
                support(a, N),
                confidence(a, c),
                lift(a, b, c, N),
                conviction(a, b, c, d, N),
            ]
        )
    elif problema == 1:
        return np.array(
            [
                casual_support(a, d, N),
                casual_confidence(a, b, c, d),
                max_confidence(a, b, c),
            ]
        )
    elif problema == 2:
        return np.array(
            [
                jaccard(a, b, c, N),
                cosine(a, b, c, N),
                phi_coefficient(a, b, c, N),
                kappa(a, b, c, d, N),
            ]
        )